#Telecom Domain Read & Write Ops Hackathon - Building Datalake & Lakehouse
This notebook contains assignments to practice Spark read options and Databricks volumes. <br>
Sections: Sample data creation, Catalog & Volume creation, Copying data into Volumes, Path glob/recursive reads, toDF() column renaming variants, inferSchema/header/separator experiments, and exercises.<br>

![](https://fplogoimages.withfloats.com/actual/68009c3a43430aff8a30419d.png)
![](https://theciotimes.com/wp-content/uploads/2021/03/TELECOM1.jpg)

##First Import all required libraries & Create spark session object

##1. Write SQL statements to create:
1. A catalog named telecom_catalog_assign
2. A schema landing_zone
3. A volume landing_vol
4. Using dbutils.fs.mkdirs, create folders:<br>
/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/
/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/
/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/
5. Explain the difference between (Just google and understand why we are going for volume concept for prod ready systems):<br>
a. Volume vs DBFS/FileStore<br>
b. Why production teams prefer Volumes for regulated data<br>

Volumes (like Unity Catalog Volumes) are modern, fine-grained storage for non-tabular data in Databricks, offering robust Unity Catalog governance (access control, lineage, lifecycle) over cloud storage, replacing older, less secure methods like DBFS (Databricks File System mounts/root) which lacked unified control, making Volumes superior for production, especially for regulated data needing strict security, audit trails, and simplified data management within the data lakehouse. 
<br><br/>

#**a. Volumes vs. DBFS/FileStore**

###**DBFS/FileStore (Legacy):**
Essentially a distributed file system (DFS) abstraction over cloud storage (S3, ADLS).<br>
DBFS Root/Mounts: Older ways to access data; mounts provided direct access to cloud storage paths, but governance was weak, often bypassed Unity Catalog, and lacked fine-grained permissions.<br>

###**Lack of Governance:**
Access was often broad (ACLs on storage level), making fine-grained control difficult and auditing challenging.<br>

###**Volumes (Modern):**
 A Unity Catalog feature providing a managed storage layer for non-tabular data (files like JSON, CSV, Parquet, images, etc.).<br>
Unified Governance: Volumes live within Unity Catalog, inheriting its powerful access control (row/column level on tables, but volumes control file access), lineage, and auditing capabilities.<br>

###**Managed vs. External:**

**Managed Volumes:**
Data stored in Unity Catalog's managed storage, lifecycle fully controlled by UC (e.g., 7-day retention option).
**External Volumes:** Register existing cloud storage paths, allowing mixed access (Databricks & external tools), with UC governing access.
Why Better: Replaces messy DBFS paths with structured Volumes/<catalog>/<schema>/<volume-name> paths, improving security, discoverability, and manageability for production. <br>

#**b. Why Production Teams Prefer Volumes for Regulated Data**

###***Granular Access Control:***
Unity Catalog enables fine-grained permissions on volumes (who can read/write specific files/folders), crucial for PII, financial, or sensitive data, unlike broad DBFS permissions.

###***Auditing & Lineage:***
All file operations on volumes are logged within Unity Catalog, providing audit trails and data lineage (tracking data movement) required for compliance (GDPR, HIPAA, etc.).

###***Data Lifecycle Management:***
Managed volumes offer automated data retention policies (e.g., deleting old logs), simplifying compliance with data retention mandates.

###***Simplified Security:***
Centralized security model via UC avoids scattered storage-level ACLs (Access Control Lists), reducing risk and simplifying audits.

###**Data Discovery:***
Volumes are registered in the Data Catalog, making sensitive files discoverable and manageable, not hidden in opaque DBFS paths.
Isolation & Security: Better isolation between workloads, preventing accidental data exposure common with legacy DBFS mounts. <br>
In essence, Volumes bring the robust governance of Unity Catalog to unstructured and semi-structured files, making them production-ready, secure, and compliant for regulated industries, a significant upgrade from the older DBFS system

In [0]:
%sql
create catalog if not exists telecom_catalog_assign;
create schema if not exists telecom_catalog_assign.landing_zone;
create volume if not exists telecom_catalog_assign.landing_zone.landing_vol;


In [0]:
dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer")
dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage")
dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower")

##Data files to use in this usecase:
customer_csv = '''
101,Arun,31,Chennai,PREPAID
102,Meera,45,Bangalore,POSTPAID
103,Irfan,29,Hyderabad,PREPAID
104,Raj,52,Mumbai,POSTPAID
105,,27,Delhi,PREPAID
106,Sneha,abc,Pune,PREPAID
'''

usage_tsv = '''customer_id\tvoice_mins\tdata_mb\tsms_count
101\t320\t1500\t20
102\t120\t4000\t5
103\t540\t600\t52
104\t45\t200\t2
105\t0\t0\t0
'''

tower_logs_region1 = '''event_id|customer_id|tower_id|signal_strength|timestamp
5001|101|TWR01|-80|2025-01-10 10:21:54
5004|104|TWR05|-75|2025-01-10 11:01:12
'''

In [0]:
customer_csv = '''
101,Arun,31,Chennai,PREPAID
102,Meera,45,Bangalore,POSTPAID
103,Irfan,29,Hyderabad,PREPAID
104,Raj,52,Mumbai,POSTPAID
105,,27,Delhi,PREPAID
106,Sneha,abc,Pune,PREPAID
'''

usage_tsv = '''
customer_id\tvoice_mins\tdata_mb\tsms_count
101\t320\t1500\t20
102\t120\t4000\t5
103\t540\t600\t52
104\t45\t200\t2
105\t0\t0\t0
'''

tower_logs_region1 = '''
event_id|customer_id|tower_id|signal_strength|timestamp
5001|101|TWR01|-80|2025-01-10 10:21:54
5004|104|TWR05|-75|2025-01-10 11:01:12
'''


dbutils.fs.put("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer.csv", customer_csv, True)
dbutils.fs.put("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage.tsv", usage_tsv, True) 
dbutils.fs.put("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower_logs_region1.csv", tower_logs_region1, True)


##2. Filesystem operations
1. Write dbutils.fs code to copy the above datasets into your created Volume folders:
Customer → /Volumes/.../customer/
Usage → /Volumes/.../usage/
Tower (region-based) → /Volumes/.../tower/region1/ and /Volumes/.../tower/region2/

2. Write a command to validate whether files were successfully copied

In [0]:
dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region1")
dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region2")

In [0]:
dbutils.fs.cp("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer.csv", "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv")
dbutils.fs.cp("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage.tsv", "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.tsv")
dbutils.fs.cp("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower_logs_region1.csv", "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region1/tower_logs_region1.csv")

In [0]:
dbutils.fs.ls("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region1/")

In [0]:
dbutils.fs.ls("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/")

In [0]:
dbutils.fs.ls("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/")


##3. Spark Directory Read Use Cases
1. Read all tower logs using:
Path glob filter (example: *.csv)
Multiple paths input
Recursive lookup

2. Demonstrate these 3 reads separately:
Using pathGlobFilter
Using list of paths in spark.read.csv([path1, path2])
Using .option("recursiveFileLookup","true")

3. Compare the outputs and understand when each should be used.

In [0]:
tower_logs_region2 = '''
event_id|customer_id|tower_id|signal_strength|timestamp
5002|102|TWR02|-80|2025-02-10 10:21:54
5003|103|TWR03|-75|2025-02-10 11:01:12
'''
dbutils.fs.put("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower_logs_region2.csv", tower_logs_region2, True)
dbutils.fs.cp("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower_logs_region2.csv", "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region2/tower_logs_region2.csv")

In [0]:
df1=spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower_logs_region*", header=True,sep='|')
display(df1)

In [0]:
df2=spark.read.csv(path=["/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower_logs_region1.csv","/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower_logs_region2.csv"], header=True,sep='|')
display(df2)

In [0]:
df3=spark.read.csv(path="/Volumes/telecom_catalog_assign/landing_zone/landing_vol/", header=True,sep='|',pathGlobFilter='t*.csv')
display(df3)

In [0]:
df4=spark.read.csv(path="/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/", header=True,sep='|',pathGlobFilter='*.csv',recursiveFileLookup=True)
display(df4)

##4. Schema Inference, Header, and Separator
1. Try the Customer, Usage files with the option and options using read.csv and format function:<br>
header=false, inferSchema=false<br>
or<br>
header=true, inferSchema=true<br>
2. Write a note on What changed when we use header or inferSchema  with true/false?<br>
3. How schema inference handled “abc” in age?<br>

In [0]:
cust_df = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv", header=False,inferSchema=False)
cust_df.printSchema()
display(cust_df)
#display(cust_df.where('_c2'=='abc'))

cust_df = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv", header=True,inferSchema=True)
cust_df.printSchema()
display(cust_df)
#display(cust_df.where('31'=='abc'))



In [0]:
usg_df = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.tsv", header=False,inferSchema=False)
usg_df.printSchema()
display(usg_df)


usg_df = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.tsv", header=True,inferSchema=True,sep='\\t')
usg_df.printSchema()
display(usg_df)


##5. Column Renaming Usecases
1. Apply column names using string using toDF function for customer data
2. Apply column names and datatype using the schema function for usage data
3. Apply column names and datatype using the StructType with IntegerType, StringType, TimestampType and other classes for towers data 

In [0]:
cust_df = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv", header=True,inferSchema=True).toDF("id","bname","age","location","type")
cust_df.printSchema()
display(cust_df)

In [0]:
cust_schema ="id integer, name string, age integer, location string, type string"
cust_df = spark.read.schema(cust_schema).csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv", header=True,inferSchema=True)
cust_df.printSchema()
display(cust_df)

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

cust_struct_typec = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("location", StringType(), True),
    StructField("type", StringType(), True)
])
cust_df = spark.read.schema(cust_struct_typec).csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv" )
cust_df.printSchema()
display(cust_df)



## Spark Write Operations using 
- csv, json, orc, parquet, delta, saveAsTable, insertInto, xml with different write mode, header and sep options

##6. Write Operations (Data Conversion/Schema migration) – CSV Format Usecases
1. Write customer data into CSV format using overwrite mode
2. Write usage data into CSV format using append mode
3. Write tower data into CSV format with header enabled and custom separator (|)
4. Read the tower data in a dataframe and show only 5 rows.
5. Download the file into local from the catalog volume location and see the data of any of the above files opening in a notepad++.

##7. Write Operations (Data Conversion/Schema migration)– JSON Format Usecases
1. Write customer data into JSON format using overwrite mode
2. Write usage data into JSON format using append mode and snappy compression format
3. Write tower data into JSON format using ignore mode and observe the behavior of this mode
4. Read the tower data in a dataframe and show only 5 rows.
5. Download the file into local harddisk from the catalog volume location and see the data of any of the above files opening in a notepad++.

##8. Write Operations (Data Conversion/Schema migration) – Parquet Format Usecases
1. Write customer data into Parquet format using overwrite mode and in a gzip format
2. Write usage data into Parquet format using error mode
3. Write tower data into Parquet format with gzip compression option
4. Read the usage data in a dataframe and show only 5 rows.
5. Download the file into local harddisk from the catalog volume location and see the data of any of the above files opening in a notepad++.

##9. Write Operations (Data Conversion/Schema migration) – Orc Format Usecases
1. Write customer data into ORC format using overwrite mode
2. Write usage data into ORC format using append mode
3. Write tower data into ORC format and see the output file structure
4. Read the usage data in a dataframe and show only 5 rows.
5. Download the file into local harddisk from the catalog volume location and see the data of any of the above files opening in a notepad++.

##10. Write Operations (Data Conversion/Schema migration) – Delta Format Usecases
1. Write customer data into Delta format using overwrite mode
2. Write usage data into Delta format using append mode
3. Write tower data into Delta format and see the output file structure
4. Read the usage data in a dataframe and show only 5 rows.
5. Download the file into local harddisk from the catalog volume location and see the data of any of the above files opening in a notepad++.
6. Compare the parquet location and delta location and try to understand what is the differentiating factor, as both are parquet files only.

##11. Write Operations (Lakehouse Usecases) – Delta table Usecases
1. Write customer data using saveAsTable() as a managed table
2. Write usage data using saveAsTable() with overwrite mode
3. Drop the managed table and verify data removal
4. Go and check the table overview and realize it is in delta format in the Catalog.
5. Use spark.read.sql to write some simple queries on the above tables created.


##12. Write Operations (Lakehouse Usecases) – Delta table Usecases
1. Write customer data using insertInto() in a new table and find the behavior
2. Write usage data using insertTable() with overwrite mode

##13. Write Operations (Lakehouse Usecases) – Delta table Usecases
1. Write customer data into XML format using rowTag as cust
2. Write usage data into XML format using overwrite mode with the rowTag as usage
3. Download the xml data and open the file in notepad++ and see how the xml file looks like.

##14. Compare all the downloaded files (csv, json, orc, parquet, delta and xml) 
1. Capture the size occupied between all of these file formats and list the formats below based on the order of size from small to big.

###15. Try to do permutation and combination of performing Schema Migration & Data Conversion operations like...
1. Read any one of the above orc data in a dataframe and write it to dbfs in a parquet format
2. Read any one of the above parquet data in a dataframe and write it to dbfs in a delta format
3. Read any one of the above delta data in a dataframe and write it to dbfs in a xml format
4. Read any one of the above delta table in a dataframe and write it to dbfs in a json format
5. Read any one of the above delta table in a dataframe and write it to another table

##16. Do a final exercise of defining one/two liner of... 
1. When to use/benifits csv
2. When to use/benifits json
3. When to use/benifit orc
4. When to use/benifit parquet
5. When to use/benifit delta
6. When to use/benifit xml
7. When to use/benifit delta tables
